In [ ]:
import pandas as pd
import json
import time
import google.generativeai as genai
import google.api_core.exceptions
import google.generativeai.types as safety_types
import os

In [ ]:
# DANH SÁCH API KEYS CHO GEMINI
# Thay thế bằng các API key thực tế của bạn
GEMINI_API_KEYS = [
    os.environ.get('GOOGLE_API_KEY')
]

# Giới hạn tỷ lệ cho mỗi API key
API_RATE_LIMIT_PER_MINUTE = 100
API_KEY_COOLDOWN_SECONDS = 70 # 1 phút

# Cấu trúc để theo dõi việc sử dụng API key
api_key_usage = {key: {"count": 0, "last_reset_time": time.time()} for key in GEMINI_API_KEYS}
current_api_key_index = 0 # Bắt đầu với API key đầu tiên

In [ ]:
def get_next_available_api_key():
    global current_api_key_index
    global api_key_usage
    start_index = current_api_key_index
    while True:
        api_key = GEMINI_API_KEYS[current_api_key_index]
        usage_stats = api_key_usage[api_key]
        current_time = time.time()
        if current_time - usage_stats["last_reset_time"] > API_KEY_COOLDOWN_SECONDS:
            usage_stats["count"] = 0
            usage_stats["last_reset_time"] = current_time
        if usage_stats["count"] < API_RATE_LIMIT_PER_MINUTE:
            usage_stats["count"] += 1
            genai.configure(api_key=api_key)
            return api_key
        current_api_key_index = (current_api_key_index + 1) % len(GEMINI_API_KEYS)
        if current_api_key_index == start_index:
            current_time = time.time()
            wait_times = []
            for key_str_loop, usage_stats_loop in api_key_usage.items():
                time_since_last_reset = current_time - usage_stats_loop["last_reset_time"]
                if time_since_last_reset > API_KEY_COOLDOWN_SECONDS:
                    wait_times.append(0)
                else:
                    wait_times.append(API_KEY_COOLDOWN_SECONDS - time_since_last_reset + 1)
            min_wait_time = min(wt for wt in wait_times if wt > 0) if any(wt > 0 for wt in wait_times) else 0.5
            
            if min_wait_time > 0:
                 print(f"All API keys are on cooldown. Waiting for {min_wait_time:.2f} seconds...")
                 time.sleep(min_wait_time)

In [ ]:
# Read the CSV files
source_df = pd.read_csv('question.csv')
target_df = pd.read_csv('result_QP.csv') # Giữ nguyên tên file target của bạn

# Ensure both DataFrames have the same number of rows
assert len(source_df) == len(target_df)

In [ ]:
def check_column_mapping():
    results_validation = []
    model_name_to_use = 'gemini-2.0-flash' 
    print(f"Using LLM model: {model_name_to_use}")

    # CẬP NHẬT VALID_LLM_OUTPUTS
    VALID_LLM_OUTPUTS = {
        "Wrong data selection - Missing column",
        # "Wrong data selection - Missing row", # Đã xóa
        # "Wrong data selection - Extra row",   # Đã xóa
        "Wrong data selection - Different content",
        "Extra column",
        "True"
    }
    MAX_RETRIES_PER_RECORD = len(GEMINI_API_KEYS) + 1 if GEMINI_API_KEYS else 1

    for i in range(len(source_df)):
        expected_result_str = source_df.loc[i, 'Result']
        actual_result_str = target_df.loc[i, 'Result']
        
        print(f"\n--- Record {i} ---")

        # PROMPT ĐÃ ĐƯỢC CẬP NHẬT
        prompt = f"""
You are an SQL result verifier. Your task is to compare an 'expected_result' (source) with an 'actual_result' (target) and determine the validation status.
The results are provided as JSON strings representing lists of objects.
**Crucial Assumption:** Assume all objects within a single JSON result list (either expected or actual) have the same set of keys. If a list is empty (e.g., `[]`), it has no keys and no rows. If a list contains empty objects (e.g., `[ {{}} ]`), it has rows, but each object has an empty set of keys.

Expected Result (Source):
```json
{expected_result_str}
```

Actual Result (Target):
```json
{actual_result_str}
```

Based on the following rules, determine the validation status. Respond with *only one* of these exact strings:
- 'Wrong data selection - Missing column'
- 'Wrong data selection - Different content'
- 'Extra column'
- 'True'

**Definitions:**
Let `expected_data` be the list of objects parsed from Expected Result.
Let `actual_data` be the list of objects parsed from Actual Result.

To determine `expected_keys` (as a set):
- If `expected_data` is an empty list (e.g., `[]`), then `expected_keys` is an empty set.
- Else (if `expected_data` is not empty, e.g., `[ {{}}]` or `[{{"a":1}}]`), then `expected_keys` is the set of keys from the first object `expected_data[0]`. (If `expected_data[0]` is `{{}}`, `expected_keys` is an empty set).
Determine `actual_keys` (as a set) using the same logic with `actual_data`.

**RULES (evaluate strictly in the order presented. Once a rule's conditions are met, select its corresponding validation status and do not proceed to subsequent rules.):**

1.  **'Wrong data selection - Missing column':**
    Apply if `expected_keys` is not empty AND (`actual_keys` is empty OR `expected_keys` IS NOT a subset of `actual_keys`).
    *This means actual_result is missing one or more columns that are present in expected_result.*
    *Example:* Expected `[{{"a":1, "b":2}}]`, Actual `[{{"a":1}}]` -> 'Wrong data selection - Missing column' (missing 'b').
    *Example:* Expected `[{{"a":1}}]`, Actual `[{{}}]` -> 'Wrong data selection - Missing column' (missing 'a').
    *Example:* Expected `[{{"a":1}}]`, Actual `[]` -> 'Wrong data selection - Missing column' (expected column 'a', actual has no columns due to no rows).

2.  **'Extra column':**
    Apply if Rule 1 does NOT apply, AND
    (`expected_keys` IS a subset of `actual_keys` OR `expected_keys` is empty) AND `actual_keys` IS a proper superset of `expected_keys` (i.e., `actual_keys` contains all keys from `expected_keys` AND `actual_keys` has additional keys not in `expected_keys`).
    *This means actual_result has all expected columns plus one or more extra columns.*
    *Example:* Expected `[{{"a":1}}]`, Actual `[{{"a":1, "b":2}}]` -> 'Extra column'.
    *Example:* Expected `[]`, Actual `[{{"a":1}}]` -> 'Extra column'.
    *Example:* Expected `[{{}}]`, Actual `[{{"a":1}}]` -> 'Extra column'.

    *If execution reaches this point, it implies: `expected_keys` and `actual_keys` are identical sets (could both be empty). The only remaining differences can be in the number of rows or the content of the data objects, or if they are perfectly True.*

3.  **'Wrong data selection - Different content':**
    Apply if Rules 1 and 2 do NOT apply, AND (`len(actual_data) != len(expected_data)` OR `expected_data` IS NOT identical to `actual_data`).
    To check for non-identical data when row counts are the same: Compare `expected_data[i]` with `actual_data[i]` for each row `i`. For each object pair, compare all key-value pairs (since keys are now known to be identical). If any value differs for any key in any row, or if the order of rows is different such that `expected_data[i]` does not match `actual_data[i]`, the content is different.
    *This means column structure is compatible (Rule 1 and 2 did not apply), but either the number of rows is different, or the number of rows is the same but the data values within the rows differ (or row order is effectively different).*
    *Example (different row count - missing):* Expected `[{{"a":1}}, {{"a":2}}]`, Actual `[{{"a":1}}]` -> 'Wrong data selection - Different content'.
    *Example (different row count - extra):* Expected `[{{"a":1}}]`, Actual `[{{"a":1}}, {{"a":2}}]` -> 'Wrong data selection - Different content'.
    *Example (different row count - expected empty, actual has rows):* Expected `[]`, Actual `[{{}}]` -> 'Wrong data selection - Different content'.
    *Example (same row count, different values):* Expected `[{{"a":1, "b":2}}]`, Actual `[{{"a":1, "b":3}}]` -> 'Wrong data selection - Different content'.
    *Example (same row count, different row order):* Expected `[{{"a":1}}, {{"a":2}}]`, Actual `[{{"a":2}}, {{"a":1}}]` -> 'Wrong data selection - Different content'.

4.  **'True':**
    Apply if Rules 1, 2, and 3 do NOT apply.
    *This means `expected_keys` and `actual_keys` are identical, `len(actual_data)` and `len(expected_data)` are identical, AND `expected_data` IS identical to `actual_data` (all corresponding objects and their key-value pairs match in the same order).*
    *Example:* Expected `[{{"a":1, "b":2}}]`, Actual `[{{"a":1, "b":2}}]` -> 'True'.
    *Example:* Expected `[]`, Actual `[]` -> 'True'.
    *Example:* Expected `[{{}}]`, Actual `[{{}}]` -> 'True'.

Validation Status:
"""
        
        llm_verdict = "API Error" 
        current_attempt = 0
        prompt_processed_successfully = False

        while current_attempt < MAX_RETRIES_PER_RECORD and not prompt_processed_successfully:
            current_call_api_key_info = "N/A"
            selected_api_key = None
            current_attempt += 1
            
            try:
                selected_api_key = get_next_available_api_key()
                if not selected_api_key:
                    print(f"    Attempt {current_attempt}/{MAX_RETRIES_PER_RECORD}: No API key available even after waiting. Marking record {i} as API Error.")
                    llm_verdict = "API Error (No Key)" 
                    break 

                current_call_api_key_info = f"...{selected_api_key[-4:]}"
                print(f"    Attempt {current_attempt}/{MAX_RETRIES_PER_RECORD} for record {i} (Key: {current_call_api_key_info})")
                
                model = genai.GenerativeModel(model_name=model_name_to_use)
                
                current_safety_settings = [
                    {
                        "category": safety_types.HarmCategory.HARM_CATEGORY_HARASSMENT,
                        "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
                    },
                    {
                        "category": safety_types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                        "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
                    },
                    {
                        "category": safety_types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                        "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
                    },
                    {
                        "category": safety_types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                        "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
                    },
                ]
                
                response = model.generate_content(
                    contents=prompt, 
                    safety_settings=current_safety_settings
                )
                
                if hasattr(response, 'text') and response.text:
                    raw_verdict = response.text.strip()
                    if raw_verdict in VALID_LLM_OUTPUTS:
                        llm_verdict = raw_verdict
                        prompt_processed_successfully = True 
                    else:
                        print(f"    LLM returned an unexpected response: '{raw_verdict}'. Treating as an error for this attempt.")
                        llm_verdict = "LLM Format Error" 
                else:
                    if hasattr(response, 'prompt_feedback') and response.prompt_feedback and hasattr(response.prompt_feedback, 'block_reason') and response.prompt_feedback.block_reason:
                        print(f"    LLM call blocked for record {i} (Key: {current_call_api_key_info}). Reason: {response.prompt_feedback.block_reason}. Treating as an error for this attempt.")
                    else:
                        candidate_text = ""
                        try:
                            if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
                               candidate_text = response.candidates[0].content.parts[0].text if response.candidates[0].content.parts[0].text else ""
                        except (IndexError, AttributeError):
                            pass 
                            
                        if candidate_text:
                            raw_verdict = candidate_text.strip()
                            if raw_verdict in VALID_LLM_OUTPUTS:
                                llm_verdict = raw_verdict
                                prompt_processed_successfully = True
                            else:
                                print(f"    LLM returned unexpected text in candidate parts: '{raw_verdict}'. Treating as error.")
                                llm_verdict = "LLM Format Error (Candidate)"
                        else:
                            print(f"    LLM returned no text and no block reason for record {i} (Key: {current_call_api_key_info}). Treating as an error for this attempt.")
                    if not prompt_processed_successfully:
                         llm_verdict = "LLM No Response" 

                if prompt_processed_successfully:
                    break 

            except Exception as e:
                is_retryable_error = False
                error_message = str(e).lower()

                if isinstance(e, google.api_core.exceptions.ResourceExhausted) or "429" in error_message:
                    is_retryable_error = True
                    print(f"      Error (429 Resource Exhausted) on attempt {current_attempt} for record {i} with key {current_call_api_key_info}. Details: {e}")
                elif isinstance(e, google.api_core.exceptions.ServiceUnavailable) or "503" in error_message:
                    is_retryable_error = True
                    print(f"      Error (Service Unavailable/503) on attempt {current_attempt} for record {i} with key {current_call_api_key_info}. Details: {e}")
                elif isinstance(e, google.api_core.exceptions.DeadlineExceeded):
                    is_retryable_error = True
                    print(f"      Error (DeadlineExceeded) on attempt {current_attempt} for record {i} with key {current_call_api_key_info}. Details: {e}")
                elif isinstance(e, google.api_core.exceptions.InternalServerError) or "500" in error_message :
                    is_retryable_error = True
                    print(f"      Error (InternalServerError/500) on attempt {current_attempt} for record {i} with key {current_call_api_key_info}. Details: {e}")
                else:
                    print(f"      Non-retryable error on attempt {current_attempt} for record {i} with key {current_call_api_key_info}: {type(e).__name__} - {e}")
                
                llm_verdict = "API Error" 

                if is_retryable_error and current_attempt < MAX_RETRIES_PER_RECORD:
                    print(f"      Retrying prompt for record {i}...")
                else: 
                    if not is_retryable_error:
                        print(f"      Not retrying for record {i} due to non-retryable error.")
                    else:
                        print(f"      Max retries ({MAX_RETRIES_PER_RECORD}) reached for record {i}.")
                    break 
        
        if not prompt_processed_successfully and llm_verdict not in VALID_LLM_OUTPUTS :
            print(f"    Failed to get a valid LLM response for record {i} after {current_attempt} attempts. Final verdict: {llm_verdict}")
        
        results_validation.append(llm_verdict)
        print(f"  LLM Validation for Record {i}: {llm_verdict}")
        
    return results_validation

In [ ]:
results = check_column_mapping()
target_df['Validation'] = results

In [ ]:
target_df.to_csv('valid_QP_updated_rules.csv', index=False) # Đổi tên file output để không ghi đè
print("Đã ghi kết quả vào file 'valid_QP_updated_rules.csv' với cột 'Validation'.")